In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, LSTM, Dense, BatchNormalization, Dropout
from keras.models import Model
from matplotlib import pyplot as plt
from IPython.display import clear_output
from modules.lstm_decoder import LSTMDecoder
from modules.plot_losses import PlotLosses

## Data

In [ ]:
features_train = np.load('./features/lstm/encoded/features_train.npy')
features_dev = np.load('./features/lstm/encoded/features_dev.npy')
indices_train = np.load('./features/lstm/2_steps/indices_train.npy')
indices_dev = np.load('./features/lstm/2_steps/indices_dev.npy')
classes_train = np.load('./features/res_net/classes_train_make-model.npy')[indices_train]
classes_dev = np.load('./features/res_net/classes_dev_make-model.npy')[indices_dev]

In [ ]:
print(features_train.shape)
print(features_dev.shape)
print(classes_train.shape)
print(classes_dev.shape)

## Training

In [ ]:
decoder = LSTMDecoder(features_train, classes_train, features_dev, classes_dev, weights_path='./convnet_weights/lstm/decoder_lstm/weights.36-1.27.hdf5')

In [ ]:
plot_losses = PlotLosses()
checkpoint = keras.callbacks.ModelCheckpoint(
    './convnet_weights/lstm/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
    monitor='val_loss',
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='auto', 
    period=1,
)

In [ ]:
decoder.train(100, [plot_losses, checkpoint])

## LSTM

In [ ]:
inputs = Input(shape=input_shape)
X = BatchNormalization()(inputs)
X = LSTM(1024)(X)
outputs = Dense(output_shape, activation='softmax')(X)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

## Dense

In [ ]:
features_train_dense = np.concatenate([features_train[:, 0, :], features_train[:, 1, :]], axis=1)
features_dev_dense = np.concatenate([features_dev[:, 0, :], features_dev[:, 1, :]], axis=1)

In [ ]:
inputs = Input(shape=[features_train_dense.shape[1]])
X = BatchNormalization()(inputs)
X = Dense(2048, activation='relu')(X)
outputs = Dense(output_shape, activation='softmax')(X)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
model.compile(
    optimizer=tf.train.AdamOptimizer(0.000001),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

model.load_weights('./convnet_weights/lstm/weights.37-1.27.hdf5')

In [ ]:
model.fit(
    features_train_dense,
    classes_train,
    batch_size=128,
    epochs=100,
    verbose=True,
    callbacks=[plot_losses, checkpoint],
    validation_data=(features_dev_dense, classes_dev)
)